In [1]:
import inseq

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from inseq.utils.typing import SingleScorePerStepTensor
from transformers.modeling_outputs import CausalLMOutput, Seq2SeqLMOutput
from inseq.attr import StepFunctionArgs

from typing import Union

In [3]:
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf", load_in_8bit=True, device_map="auto",cache_dir="/work/qvr905/.cache/huggingface")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/qvr905/.conda/envs/myinseq/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/qvr905/.conda/envs/myinseq/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/qvr905/.conda/envs/myinseq

In [4]:
def logit_lens_fn(args: StepFunctionArgs, layer_idx: int) -> SingleScorePerStepTensor:
    """Compute the logit lens of the predicted token at the specified layer."""
    if args.is_attributed_fn:
        raise ValueError("This step function cannot be used as attribution target.")
    forward_output: Union[CausalLMOutput, Seq2SeqLMOutput] = args.attribution_model.get_forward_output(
        args.attribution_model.formatter.convert_args_to_batch(args),
        output_hidden_states=True,
    )
    if args.attribution_model.is_encoder_decoder:
        layer_last_state = forward_output.decoder_hidden_states[layer_idx][:, -1, :]
    else:
        layer_last_state = forward_output.hidden_states[layer_idx][:, -1, :]

    logits = args.attribution_model.model.lm_head(layer_last_state)[:, args.target_ids]
    return logits

In [5]:
model = inseq.load_model(model, "saliency")
inseq.register_step_function(fn=logit_lens_fn, identifier="logit_lens", overwrite=True)

The model is loaded in 8bit mode. The device cannot be changed after loading the model.


In [6]:
class TokenWithId:
    def __init__(self, token):
        self.token = token

class FeatureAttributionOutput:
    def __init__(self, sequence_attributions, step_attributions, info):
        self.sequence_attributions = sequence_attributions
        self.step_attributions = step_attributions
        self.info = info

class FeatureAttributionSequenceOutput:
    def __init__(self, source, target, source_attributions, target_attributions, step_scores, sequence_scores, attr_pos_start, attr_pos_end):
        self.source = source
        self.target = target
        self.source_attributions = source_attributions
        self.target_attributions = target_attributions
        self.step_scores = step_scores
        self.sequence_scores = sequence_scores
        self.attr_pos_start = attr_pos_start
        self.attr_pos_end = attr_pos_end

In [84]:
out = model.attribute("Super Mario Land is a game that developed by Nintendo", step_scores=["logit_lens"], layer_idx=25,method="dummy")
# Extracting target list
target_list = out.sequence_attributions[0].target
# Concatenating tokens from TokenWithId objects
target_string = ' '.join(token.token[1:] if token.token.startswith('▁') else token.token for token in target_list)
print(target_string)
#out.show()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Attributing with dummy...: 100%|██████████| 20/20 [00:02<00:00,  2.84it/s]

<s> Super Mario Land is a game that developed by N intendo for the Game Boy hand h eld console


In [9]:
layersOutputs = []
for layer_idx in range(-1, -32, -1):
    out = model.attribute("Super Mario Land is a game that developed by Nintendo", step_scores=["logit_lens"], layer_idx=layer_idx, method="dummy")
    #print(out)
    target_list = out.sequence_attributions[0].target
    target_string = ' '.join(token.token[1:] if token.token.startswith('▁') else token.token for token in target_list)
    layersOutputs.append(target_string)
    #print(target_string)
    out.show()

Attributing with dummy...: 100%|██████████| 20/20 [00:02<00:00,  2.74it/s]


,▁for,▁the,▁Game,▁Boy,▁in,▁,1,9
logit_lens,17.641,22.156,20.594,23.453,19.109,23.297,24.172,27.016


Attributing with dummy...: 100%|██████████| 20/20 [00:02<00:00,  2.73it/s]


,▁for,▁the,▁Game,▁Boy,▁in,▁,1,9
logit_lens,12.867,15.594,12.805,11.609,16.203,23.328,18.922,1.947


Attributing with dummy...: 100%|██████████| 20/20 [00:02<00:00,  2.73it/s]


...

,▁for,▁the,▁Game,▁Boy,▁in,▁,1,9
logit_lens,0.13,0.253,0.025,0.173,-0.041,0.008,0.129,0.063


Attributing with dummy...: 100%|██████████| 20/20 [00:02<00:00,  2.72it/s]


,▁for,▁the,▁Game,▁Boy,▁in,▁,1,9
logit_lens,0.119,0.231,0.002,0.156,-0.112,-0.022,0.091,0.061


Attributing with dummy...: 100%|██████████| 20/20 [00:02<00:00,  2.72it/s]


,▁for,▁the,▁Game,▁Boy,▁in,▁,1,9
logit_lens,0.01,0.131,0.015,0.091,0.132,-0.025,0.072,0.039
